## D7041E - Mini project: Human Real-time Detection

### Group: Laura Bermejo, Lina Borg, Julie Labbé

#### References and sources
For the training of our model, we got inspired by this video: https://www.youtube.com/watch?v=V4Kkrz__hvo.

#### Requirements
The required libraries are super-gradient (for model), roboflow (for dataset) and supervision (for bounding boxes).

In [16]:
%pip install -q super-gradients
%pip install -q roboflow
%pip install -q supervision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.15 requires pyparsing==2.4.7, but you have pyparsing 2.4.5 which is incompatible.
fuzzingbook 1.1 requires pyparsing==2.4.7, but you have pyparsing 2.4.5 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
super-gradients 3.5.0 requires pyparsing==2.4.5, but you have pyparsing 2.4.7 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


#### Dataset

##### Code to download the dataset

In [17]:
from roboflow import Roboflow
rf = Roboflow(api_key="DNm47rTel6WBxKZqAd9T")
project = rf.workspace("capricon").project("human-detection-q0nit")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Human-Detection-2 in yolov5pytorch::   2%|▏         | 6/376 [00:00<00:06, 59.77it/s]

Extracting Dataset Version Zip to Human-Detection-2 in yolov5pytorch:: 100%|██████████| 376/376 [00:02<00:00, 173.19it/s]


#### class Config Class definition

In [18]:
from typing import List, Dict

class config:
  #Project paths
  DATA_DIR: str = "Human-Detection-2"
  CHECKPOINT_DIR: str = "/checkpoints"
  EXPERIMENT_NAME: str = "Project"

  #Datasets
  TRAIN_IMAGES_DIR: str ="train/images"
  TRAIN_LABELS_DIR: str = "train/labels"
  VAL_IMAGES_DIR: str = "valid/images"
  VAL_LABELS_DIR: str = "valid/labels"
  TEST_IMAGES_DIR: str = "test/images"
  TEST_LABELS_DIR: str = "test/labels"

  #Classes
  CLASSES: List[str] = ["human", "vehicle"]
  NUM_CLASSES: int = 2

  #Model 1
  # DEFINE HYPERPARAMETERS, YOU WILL HAVE TO CHANGE IT
  DATALOADER_PARAMS: Dict = {
      'batch_size':16,
      'num_workers':0
  }
  # THIS IS ALREADY SET, CHANGE ONLY DATALOADER PARAMS
  MODEL_NAME: str = 'yolo_nas_l'
  PRETRAINED_WEIGHTS: str ='coco'

#### Dataloaders initialization

In [13]:
from IPython.display import clear_output
from typing import List, Dict
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_val
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from typing import List, Dict
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_val

In [14]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

[2024-01-06 21:47:15] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 159/159 [00:00<00:00, 681.92it/s]
[2024-01-06 21:47:15] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 159/159 [00:00<00:00, 515.85it/s]
[2024-01-06 21:47:16] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:  43%|████▎     | 69/159 [00:00<00:00, 685.50it/s]

Indexing dataset annotations: 100%|██████████| 159/159 [00:00<00:00, 503.70it/s]
